# Building Agents with memory 

## Preparation

In [ ]:
#!rm  -f ~/.letta/sqlite.db

## client setup

In [ ]:
from helper import nb_print  # more legible printing

In [ ]:
from letta import create_client 

client = create_client() 

In [ ]:
from letta import EmbeddingConfig, LLMConfig

client.set_default_embedding_config(EmbeddingConfig.default_config(provider="openai"))
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## 1. a simple agent with memory 

### creating an agent 

In [ ]:
agent_name = "simple_agent"

In [ ]:
# this deletes the agent if you run this a second time
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sanika", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

In [ ]:
response.usage

In [ ]:
response

In [ ]:
nb_print(response.messages)

### agent state 

In [ ]:
print(agent_state.system)

In [ ]:
agent_state.tools

### Viewing an agent's memory

In [ ]:
agent_state.memory

In [ ]:
client.get_archival_memory_summary(agent_state.id)

In [ ]:
client.get_recall_memory_summary(agent_state.id)

In [ ]:
client.get_messages(agent_state.id)[0]

## 2. core memory 

### core memories about the human

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Max Mustermann.", 
    role = "user"
) 
nb_print(response.messages)

### Memories about the agent

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate.", 
    role = "user"
) 

In [ ]:
nb_print(response.messages)

In [ ]:
client.get_core_memory(agent_state.id).get_block('persona')

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Oops, I was wrong. My name is actually Sanika", 
    role = "user"
) 
nb_print(response.messages)

## 3. archival memory

In [ ]:
client.get_archival_memory(agent_state.id)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'Sanika loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

In [ ]:
client.get_archival_memory(agent_state.id)[0].text

In [ ]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Sanika loves Beagles"
)

In [ ]:
passage  # under the hood, it's all embeddings

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)